In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from pmdarima.arima import auto_arima
from fbprophet import Prophet
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
url = 'https://github.com/neylsoncrepalde/projeto_eda_covid/blob/master/covid_19_data.csv?raw=true'
df = pd.read_csv(url, parse_dates=['ObservationDate', 'Last Update'])

In [ ]:
#Função para correção de colunas
def corrige_colunas(col_name):
    return re.sub(r"[/| ]", "", col_name).lower()

df.columns = [corrige_colunas(col) for col in df.columns]

In [ ]:
# Análise exploratória dos casos no Brasil
brasil = df.loc[(df.countryregion == 'Brazil') & (df.confirmed > 0)]

In [ ]:
# Gráfico de casos confirmados
fig1 = px.line(brasil, 'observationdate', 'confirmed', 
               labels={'observationdate':'Data', 'confirmed':'Número de casos confirmados'},
               title='Casos confirmados no Brasil')


In [ ]:
# Gráfico de novos casos por dia
brasil['novoscasos'] = list(map(
    lambda x: 0 if (x==0) else brasil['confirmed'].iloc[x] - brasil['confirmed'].iloc[x-1],
    np.arange(brasil.shape[0])
))

fig2 = px.line(brasil, x='observationdate', y='novoscasos', title='Novos casos por dia',
               labels={'observationdate': 'Data', 'novoscasos': 'Novos casos'})

In [ ]:
# Gráfico de mortes
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=brasil.observationdate, y=brasil.deaths, name='Mortes', mode='lines+markers',
                          line=dict(color='red')))
fig3.update_layout(title='Mortes por COVID-19 no Brasil',
                   xaxis_title='Data',
                   yaxis_title='Número de mortes')

In [ ]:
# Função para calcular taxa de crescimento
def taxa_crescimento(data, variable, data_inicio=None, data_fim=None):
    
    # Filtrar dados no período especificado
    if data_inicio:
        data = data[data.observationdate >= data_inicio]
    if data_fim:
        data = data[data.observationdate <= data_fim]

    # Calcular a taxa de crescimento
    inicio_valor = data[variable].iloc[0]
    fim_valor = data[variable].iloc[-1]

    taxa = ((fim_valor / inicio_valor) ** (1 / len(data.index)) - 1) * 100

    return taxa

In [ ]:
# Taxa de crescimento médio
cresc_medio = taxa_crescimento(brasil, 'confirmed')
print(f"O crescimento médio do COVID no Brasil no período avaliado foi de {cresc_medio.round(2)}%.")

In [ ]:
# Função para calcular taxa de crescimento diária
def taxa_crescimento_diaria(data, variable, data_inicio=None):

    # Filtrar dados a partir da data de início, se fornecida
    if data_inicio:
        data = data[data.observationdate >= data_inicio]

    # Calcular as taxas de crescimento diárias
    taxas_diarias = (
        (data[variable].diff() / data[variable].shift(1)).fillna(0) * 100
    ).values
    
    return taxas_diarias

In [ ]:
# Gráfico da taxa de crescimento diária
fig4 = go.Figure(go.Scatter(
    x=pd.date_range(primeiro_dia, brasil.observationdate.max())[1:],
    y=tx_dia, title='Taxa de crescimento de casos confirmados no Brasil',
    labels={'y':'Taxa de crescimento', 'x':'Data'}
))

In [ ]:
# Análise de decomposição sazonal
novoscasos = brasil.novoscasos
novoscasos.index = brasil.observationdate
res = seasonal_decompose(novoscasos)

confirmados = brasil.confirmed
confirmados.index = brasil.observationdate
res2 = seasonal_decompose(confirmados)


In [ ]:
# Modelo ARIMA
modelo = auto_arima(confirmados)
pd.date_range('2020-05-01', '2020-05-19')

In [ ]:

# Gráfico da previsão de casos confirmados
fig5 = go.Figure(go.Scatter(
    x=confirmados.index, y=confirmados, name='Observed'
))
fig5.add_trace(go.Scatter(x=confirmados.index, y=modelo.predict_in_sample(), name='Predicted'))
fig5.add_trace(go.Scatter(x=pd.date_range('2020-05-20', '2020-06-05'), y=modelo.predict(15), name='Forecast'))
fig5.update_layout(title='Previsão de casos confirmados para os próximos 15 dias',
                   xaxis_title='Data',
                   yaxis_title='Número de casos confirmados')


In [ ]:
# Modelo Prophet
profeta = pd.DataFrame(zip(brasil.observationdate, brasil.confirmed), columns=['ds', 'y'])
modelo = Prophet(interval_width=0.95)
modelo.fit(profeta)


In [ ]:
# Previsão usando o Prophet
futuro = modelo.make_future_dataframe(periods=15)
previsao = modelo.predict(futuro)


In [ ]:
# Gráfico da previsão usando o Prophet
fig6 = go.Figure()
fig6.add_trace(go.Scatter(x=profeta.ds, y=profeta.y, name='Observado', mode='markers'))
fig6.add_trace(go.Scatter(x=previsao.ds, y=previsao.yhat, name='Previsto'))
fig6.update_layout(title='Previsão de casos confirmados usando o Prophet',
                   xaxis_title='Data',
                   yaxis_title='Número de casos confirmados')